In [27]:
import os
import shutil
import datetime
import concurrent.futures

In [33]:
def get_minimum_date(file_path):
    try:
        stat = os.stat(file_path)
        creation_time = stat.st_ctime
        access_time = stat.st_atime
        modified_time = stat.st_mtime

        return datetime.datetime.fromtimestamp(min(creation_time, access_time, modified_time)).date()

    except OSError as e:
        print(f"Error getting minimum date for {file_path}: {e}")
        return None

In [34]:
def organize_files_by_date(root_directory):
    for root, _, files in os.walk(root_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            minimum_date = get_minimum_date(file_path)
            if minimum_date is not None:
                year_folder = os.path.join(root_directory, str(minimum_date.year))
                destination_folder = os.path.join(year_folder, filename)

                if not os.path.exists(year_folder):
                    os.mkdir(year_folder)

                shutil.move(file_path, destination_folder)

In [ ]:
def organize_files_by_date_using_threads(root_directory):
    def move_file(file_path, destination_folder):
        shutil.move(file_path, destination_folder)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for root, _, files in os.walk(root_directory):
            for filename in files:
                file_path = os.path.join(root, filename)
                minimum_date = get_minimum_date(file_path)
                if minimum_date is not None:
                    year_folder = os.path.join(root_directory, str(minimum_date.year))
                    destination_folder = os.path.join(year_folder, filename)

                    if not os.path.exists(year_folder):
                        os.mkdir(year_folder)

                    futures.append(executor.submit(move_file, file_path, destination_folder))

        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error moving file: {e}")

In [48]:
root_directory = r"E:\Images\Rishu\Camera" # Replace this with your actual root directory path
organize_files_by_date(root_directory)

In [50]:
# Delete empty directories
try:
    import subprocess
    cmd = f'ROBOCOPY {root_directory} {root_directory} /S /MOVE'
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True, check=True)
except subprocess.CalledProcessError as e:
    #print(f"Error: {e}")
    pass

# New Features

### Categorising Only

In [87]:
import os
import shutil
import datetime
import concurrent.futures

IMAGE_EXTENSIONS = (".jpg", ".jpeg",".heic")
VIDEO_EXTENSIONS = (".mp4", ".avi", ".mkv", ".mov", ".wmv")

def get_minimum_date(file_path):
    try:
        stat = os.stat(file_path)
        creation_time = stat.st_ctime
        access_time = stat.st_atime
        modified_time = stat.st_mtime

        return datetime.datetime.fromtimestamp(min(creation_time, access_time, modified_time)).date()

    except OSError as e:
        print(f"Error getting minimum date for {file_path}: {e}")
        return None

def organize_files_by_date(root_directory):
    def move_file(file_path, destination_folder):
        shutil.move(file_path, destination_folder)

    def categorize_file(file_path, year_folder):
        _, ext = os.path.splitext(file_path)

        if ext.lower() in IMAGE_EXTENSIONS:
            media_type_folder = os.path.join(year_folder, "Images")
        elif ext.lower() in VIDEO_EXTENSIONS:
            media_type_folder = os.path.join(year_folder, "Videos")
        else:
            media_type_folder = os.path.join(year_folder, "Others")

        if not os.path.exists(media_type_folder):
            os.mkdir(media_type_folder)

        shutil.move(file_path, os.path.join(media_type_folder, os.path.basename(file_path)))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for root, _, files in os.walk(root_directory):
            for filename in files:
                file_path = os.path.join(root, filename)
                minimum_date = get_minimum_date(file_path)
                if minimum_date is not None:
                    year_folder = os.path.join(root_directory, str(minimum_date.year))
                    destination_folder = os.path.join(year_folder, filename)

                    if not os.path.exists(year_folder):
                        os.mkdir(year_folder)

                    futures.append(executor.submit(move_file, file_path, destination_folder))

        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error moving file: {e}")

    # Categorize files into Images, Videos, and Others
    for root, _, files in os.walk(root_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            minimum_date = get_minimum_date(file_path)
            if minimum_date is not None:
                year_folder = os.path.join(root_directory, str(minimum_date.year))
                categorize_file(file_path, year_folder)

if __name__ == "__main__":
    root_directory = r"E:\Images\Rishu\Camera\Trial"
    organize_files_by_date(root_directory)


### Using Multithreading for Categorising

In [ ]:
root_directory = r"E:\Images\Rishu\Camera\Trial"

In [7]:
import os
import shutil
import datetime
import argparse
import concurrent.futures

IMAGE_EXTENSIONS = (".jpg", ".jpeg",".heic")
VIDEO_EXTENSIONS = (".mp4", ".avi", ".mkv", ".mov", ".wmv")

def get_minimum_date(file_path):
    try:
        stat = os.stat(file_path)
        creation_time = stat.st_ctime
        access_time = stat.st_atime
        modified_time = stat.st_mtime

        return datetime.datetime.fromtimestamp(min(creation_time, access_time, modified_time)).date()

    except OSError as e:
        print(f"Error getting minimum date for {file_path}: {e}")
        return None

def organize_files_by_date(root_directory):
    def move_file(file_path, destination_folder):
        shutil.move(file_path, destination_folder)

    def categorize_file(file_path, year_folder):
        _, ext = os.path.splitext(file_path)

        if ext.lower() in IMAGE_EXTENSIONS:
            media_type_folder = os.path.join(year_folder, "Images")
        elif ext.lower() in VIDEO_EXTENSIONS:
            media_type_folder = os.path.join(year_folder, "Videos")
        else:
            media_type_folder = os.path.join(year_folder, "Others")

        if not os.path.exists(media_type_folder):
            os.mkdir(media_type_folder)

        shutil.move(file_path, os.path.join(media_type_folder, os.path.basename(file_path)))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for root, _, files in os.walk(root_directory):
            for filename in files:
                file_path = os.path.join(root, filename)
                minimum_date = get_minimum_date(file_path)
                if minimum_date is not None:
                    year_folder = os.path.join(root_directory, str(minimum_date.year))
                    destination_folder = os.path.join(year_folder, filename)

                    if not os.path.exists(year_folder):
                        os.mkdir(year_folder)

                    futures.append(executor.submit(move_file, file_path, destination_folder))

        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error moving file: {e}")

    # Categorize files into Images, Videos, and Others
    for root, _, files in os.walk(root_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            minimum_date = get_minimum_date(file_path)
            if minimum_date is not None:
                year_folder = os.path.join(root_directory, str(minimum_date.year))
                categorize_file(file_path, year_folder)
    
    def handle_live_photos(folder_path):
        images_folder = os.path.join(folder_path, "Images")
        videos_folder = os.path.join(folder_path, "Videos")
        live_photos_folder = os.path.join(folder_path, "Live Photos")

        for image_file in os.listdir(images_folder):
            if os.path.splitext(image_file)[0] + ".MOV" in os.listdir(videos_folder) or os.path.splitext(image_file)[0] + ".mov" in os.listdir(videos_folder):
                image_path = os.path.join(images_folder, image_file)
                video_path = os.path.join(videos_folder, os.path.splitext(image_file)[0] + ".MOV")

                if not os.path.exists(live_photos_folder):
                    os.mkdir(live_photos_folder)

                shutil.move(image_path, os.path.join(live_photos_folder, image_file))
                shutil.move(video_path, os.path.join(live_photos_folder, os.path.basename(video_path)))

    # Handle Live Photos
    
    for root, dirs, _ in os.walk(root_directory):
        for dir_name in dirs:
            folder_path = os.path.join(root_directory, dir_name)
            try:

                handle_live_photos(folder_path)
            except Exception:
                pass

if __name__ == "__main__":
    root_directory = r"E:\Images\Rishu\Camera\Trial"
    organize_files_by_date(root_directory)


# Better way to do this

In [15]:
import os
import shutil
import datetime
import concurrent.futures


In [16]:
IMAGE_EXTENSIONS = (".jpg", ".jpeg", ".heic")
VIDEO_EXTENSIONS = (".mp4", ".avi", ".mkv", ".mov", ".wmv")

In [17]:
def get_minimum_date(file_path):
    try:
        stat = os.stat(file_path)
        creation_time = stat.st_ctime
        access_time = stat.st_atime
        modified_time = stat.st_mtime

        return datetime.datetime.fromtimestamp(min(creation_time, access_time, modified_time)).date()

    except OSError as e:
        print(f"Error getting minimum date for {file_path}: {e}")
        return None


In [18]:

    
def move_and_categorize(file_path, folder_path):
    _, ext = os.path.splitext(file_path)

    if ext.lower() in IMAGE_EXTENSIONS:
        media_type_folder = os.path.join(folder_path, "Images")
    elif ext.lower() in VIDEO_EXTENSIONS:
        media_type_folder = os.path.join(folder_path, "Videos")
    else:
        media_type_folder = os.path.join(folder_path, "Others")

    if not os.path.exists(media_type_folder):
        os.mkdir(media_type_folder)

    shutil.move(file_path, os.path.join(media_type_folder, os.path.basename(file_path)))


In [19]:

def handle_live_photos(folder_path):
    images_folder = os.path.join(folder_path, "Images")
    videos_folder = os.path.join(folder_path, "Videos")
    live_photos_folder = os.path.join(folder_path, "Live Photos")

    for image_file in os.listdir(images_folder):
        if os.path.splitext(image_file)[0] + ".MOV" in os.listdir(videos_folder):
            image_path = os.path.join(images_folder, image_file)
            video_path = os.path.join(videos_folder, os.path.splitext(image_file)[0] + ".MOV")

            if not os.path.exists(live_photos_folder):
                os.mkdir(live_photos_folder)

            shutil.move(image_path, os.path.join(live_photos_folder, image_file))
            shutil.move(video_path, os.path.join(live_photos_folder, os.path.basename(video_path)))


In [20]:
def organize_files_by_date(root_directory):

    #Move files into Categories
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for root, _, files in os.walk(root_directory):
            for filename in files:
                file_path = os.path.join(root, filename)
                minimum_date = get_minimum_date(file_path)
                if minimum_date is not None:
                    year_folder = os.path.join(root_directory, str(minimum_date.year))

                    if not os.path.exists(year_folder):
                        os.mkdir(year_folder)

                    futures.append(executor.submit(move_and_categorize, file_path, year_folder))

        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error moving and categorizing file: {e}")

    # Handle Live Photos
    for root, dirs, _ in os.walk(root_directory):
        for dir_name in dirs:
            folder_path = os.path.join(root_directory, dir_name)
            try:

                handle_live_photos(folder_path)
            except Exception:
                pass


In [ ]:
def create_category_folders(root_dir):
    image_extensions = [".jpg", ".jpeg", ".png", ".gif", ".bmp"]
    video_extensions = [".mp4", ".avi", ".mov", ".mkv", ".wmv"]

    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)

            if any(file_path.lower().endswith(ext) for ext in image_extensions):
                category_folder = "Images"
            elif any(file_path.lower().endswith(ext) for ext in video_extensions):
                category_folder = "Videos"
            else:
                category_folder = "Unknown"

            source_dir = os.path.dirname(file_path)

            if os.path.basename(os.path.abspath(source_dir)) in ['Images','Videos','Unknown']:
                print("Skipped!!")
                return
            
            destination_dir = os.path.join(source_dir, category_folder)

            # Create the category folder if it doesn't exist
            if not os.path.exists(destination_dir):
                os.makedirs(destination_dir)

            # Move the file to the category folder
            shutil.move(file_path, os.path.join(destination_dir, filename))

In [26]:
root_directory = r"E:\Videos\Pankaj\Memories\Pondi"
organize_files_by_date(root_directory)
create_category_folders(root_directory)


# Delete empty directories
import subprocess

try:
    cmd = f'ROBOCOPY {root_directory} {root_directory} /S /MOVE'
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True, check=True)

except subprocess.CalledProcessError as e:
    #print(f"Error: {e}")
    pass


Only Categorise using basic stuff

In [3]:
import os
import shutil



In [5]:

root_directory = r"E:\Videos\Pankaj\Memories\Pondi"

# Delete empty directories
try:
    import subprocess
    cmd = 'ROBOCOPY . . /S /MOVE'
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True, check=True)
except subprocess.CalledProcessError as e:
    #print(f"Error: {e}")
    pass
